In [2]:
import pandas as pd
import numpy as np

In [3]:
def delete_data(df_path):
  df = pd.read_csv(df_path)
  df = df.drop(columns=['연령별(1)'])
  df = df[df['연령별(2)'] != '소계']
  df.columns = df.iloc[0]
  df = df[1:]
  df = df.reset_index(drop=True)
  new_columns = []
  for col in range(len(df.columns)):
      new_column_name = f"{df.columns[col]}_{str(df.iloc[0, col])}"
      new_columns.append(new_column_name)
  df.columns = new_columns
  df = df.drop(index=0)
  df.columns = ['연령별'] + list(df.columns[1:])
  df = df.T
  df.columns = df.iloc[0]
  df = df.drop(index=df.index[0])
  df = df.apply(pd.to_numeric, errors='coerce')
  sums = []
  for i in range(0, len(df.columns), 10):
      start = int(df.columns[i].replace('세', ''))
      end = int(df.columns[i+9].replace('세', ''))
      sum_col = df.iloc[:, i:i+10].sum(axis=1)
      sums.append(sum_col)
      sums[-1].name = f"{start}세~{end}세"
  df = pd.concat(sums, axis=1)
  df = df.T

  return df

In [14]:
df = delete_data('20210100.csv')

indices = range(0, len(df.columns), 3)
selected_columns = df.iloc[:, indices]
selected_columns

column_sums = df.sum()
column_sums.name = '합계'
df_with_total = pd.concat([df, column_sums.to_frame().T])

sum_row = df_with_total.loc[['합계']]
#sum_row = sum_row.T

sum_row.to_csv("output.csv", index=False, encoding="utf-8-sig")
print("CSV 파일이 'output.csv'로 저장되었습니다.")

CSV 파일이 'output.csv'로 저장되었습니다.
